---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    ratio = len(spam_data[spam_data['target'] == 1])/len(spam_data)
    
    return ratio*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    
    return sorted(vect.get_feature_names(),key = lambda x:len(x),reverse=True)[0]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    MultiNB = MultinomialNB(alpha = .1).fit(X_train_vectorized,y_train)
    predictions = MultiNB.predict(vect.transform(X_test))
    return roc_auc_score(y_test,predictions)

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# It seems too complicated

tfidf = TfidfVectorizer().fit(X_train)
X_train_tfidf = tfidf.transform(X_train)

feature_names = np.array(tfidf.get_feature_names())
ordered_index = X_train_tfidf.max(0).toarray()[0].argsort()

largest = pd.DataFrame(sorted(X_train_tfidf.max(0).toarray()[0],reverse=True)[:20])
largest.index = feature_names[ordered_index][:-21:-1]
largest.reset_index(inplace = True)
largest.sort_values([0,'index'],ascending=[False,True],inplace = True)
a = pd.Series(largest[0])
a.index = largest['index']
a.name = 'largest features'

smallest = pd.DataFrame(sorted(X_train_tfidf.max(0).toarray()[0],reverse = False)[:20])
smallest.index = feature_names[ordered_index][:20]
smallest.reset_index(inplace = True)
smallest.sort_values([0,'index'],ascending = [True,True],inplace=True)
b = pd.Series(smallest[0])
b.index = smallest['index']

b.name = 'smallest features'

def answer_four():
    return (b,a)

In [10]:
answer_four()

(index
 aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 Name: smallest features, dtype: float64, index
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick      

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn import naive_bayes    

tfidf = TfidfVectorizer(min_df= 3).fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
model = naive_bayes.MultinomialNB(alpha = .1).fit(X_train_tfidf,y_train)
predicted = model.predict(X_test_tfidf)


def answer_five():    
    return roc_auc_score(y_test,predicted)

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
import nltk
nltk.download('punkt')

# tokenize the text which has been joined together 
tokenized = nltk.word_tokenize(' '.join(line for line in spam_data['text'][spam_data['target'] == 1]))
tokenized_sent = spam_data['text'][spam_data['target'] == 1]
character_sum = np.sum([len(i) for i in tokenized]) # sum up lengths of all Tokens
mean_char = character_sum/len(tokenized_sent)

# same process for non-spam
tokenized = nltk.word_tokenize(' '.join(line for line in spam_data['text'][spam_data['target'] == 0]))
tokenized_sent = spam_data['text'][spam_data['target'] == 0]
character_sum = np.sum([len(i) for i in tokenized]) # sum up lengths of all Tokens
mean_char_non = character_sum/len(tokenized_sent)


def answer_six():    

    return (mean_char_non,mean_char)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
answer_six()

(57.750259067357511, 115.99866131191432)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC
tfidf = TfidfVectorizer(min_df= 5).fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# calculate length of each entry
sent_length = X_train.apply(lambda x:len(''.join(nltk.word_tokenize(x))))
sent_length_test = X_test.apply(lambda x:len(''.join(nltk.word_tokenize(x))))
X_train_tfidf = add_feature(X_train_tfidf,sent_length)
X_test_tfidf = add_feature(X_test_tfidf,sent_length_test)

# fit a SVC model
predicted = SVC(C = 1e5).fit(X_train_tfidf,y_train).predict(X_test_tfidf)

def answer_seven():
    
    return roc_auc_score(y_test,predicted)

In [17]:
answer_seven()


0.96067475340814557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
nonspam = sum([len(i) for i in nltk.word_tokenize(' '.join(line for line in spam_data['text'][spam_data['target'] == 0])) if i.isdigit()])
non_spam_len = len(spam_data['text'][spam_data['target'] == 0])
digits_non_spam = nonspam/non_spam_len

digits_spam = nonspam = sum([len(i) for i in nltk.word_tokenize(' '.join(line for line in spam_data['text'][spam_data['target'] == 1])) if i.isdigit()])/len(spam_data['text'][spam_data['target'] == 1])


def answer_eight():
    
    return (digits_non_spam,digits_spam)

In [19]:
answer_eight()

(0.18963730569948187, 9.532797858099062)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

digits_per_train = X_train.str.findall(r'\d').apply(lambda x:len(x)) # find all digits with regular expression
digits_per_test = X_test.str.findall(r'\d').apply(lambda x:len(x))

X_train_tfidf = add_feature(X_train_tfidf,sent_length) # can use a list of features instead
X_train_tfidf = add_feature(X_train_tfidf,digits_per_train)

X_test_tfidf = add_feature(X_test_tfidf,sent_length_test)
X_test_tfidf = add_feature(X_test_tfidf,digits_per_test)

lr = LogisticRegression().fit(X_train_tfidf, y_train)
pred = lr.predict(X_test_tfidf)

def answer_nine():
    
    return roc_auc_score(y_test,pred)

In [21]:
answer_nine()

0.92812759961292302

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
nonspam_special = sum([len(i) for i in spam_data['text'][spam_data['target'] == 0].apply(lambda x:''.join(nltk.word_tokenize(x))).str.findall(r'\W')])
spam_special = sum([len(i) for i in spam_data['text'][spam_data['target'] == 1].apply(lambda x:''.join(nltk.word_tokenize(x))).str.findall(r'\W')])
nonspam_special_mean = nonspam_special/len(spam_data['text'][spam_data['target'] == 0])
spam_special_mean = spam_special/len(spam_data['text'][spam_data['target'] == 1])

def answer_ten():
    
    return (nonspam_special_mean,spam_special_mean)

In [23]:
answer_ten()

(4.018445595854923, 6.174029451137885)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
vect = CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
X_train_vect = vect.transform(X_train)
X_test_vect = vect.transform(X_test)

train_special = [len(i) for i in X_train.apply(lambda x:''.join(nltk.word_tokenize(x))).str.findall(r'\W')]
test_special = [len(i) for i in X_test.apply(lambda x:''.join(nltk.word_tokenize(x))).str.findall(r'\W')]

X_train_vect = add_feature(X_train_vect,sent_length)
X_train_vect = add_feature(X_train_vect,digits_per_train)
X_train_vect = add_feature(X_train_vect,train_special)

X_test_vect = add_feature(X_test_vect,[sent_length_test,digits_per_test,test_special])

lr = LogisticRegression(C = 100).fit(X_train_vect,y_train)
pred = lr.predict(X_test_vect)

min_coefs = lr.coef_[0].argsort()[:10]
max_coefs = lr.coef_[0].argsort()[:-11:-1]

# get feature names and append three extra ones
feature_names = list(vect.get_feature_names())
for i in ['length_of_doc','digit_count','non_word_char_count']:
    feature_names.append(i)

min_features = [feature_names[i] for i in min_coefs]
max_features = [feature_names[i] for i in max_coefs]

def answer_eleven():    
    return (roc_auc_score(y_test,pred),min_features,max_features)

In [25]:
answer_eleven()

(0.97632124000475362,
 ['. ', '..', ' i', ' y', ' go', '? ', ' m', ' h', 'go', 'h '],
 ['digit_count', 'ne', 'co', 'xt', 'ar', 'ia', 'ww', ' x', 'mob', ' ch'])